## Agregacja i normalizacja danych
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne importy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import iqr, zscore

## Wczytanie danych

In [ ]:
def get_data():
    os.chdir('..')
    os.chdir('Zadanie1\\czesc4')
    read = []
    for filename in os.listdir():
        df = pd.read_csv(filename)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
        read.append(df)
    os.chdir('..')
    os.chdir('..')
    os.chdir('Zadanie2')
    return read

data = get_data()
part6 = data[5].copy()

## <center>Część 1</center>
### Wykonaj wykresy pudełkowe (ang.box plot), które pozwolą na graficzną identyfikację wartości odstających poprzez przedstawienie kwartyli i wartości skrajnych w danych dotyczących:

### 1.1 liczby nowych zachorowań

In [ ]:
sns.boxplot(data=part6, y='daily_confirmed', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych zachorowań')
plt.ylabel('Nowe zachorowania')

### 1.2 liczby nowych śmierci

In [ ]:
sns.boxplot(data=part6, y='daily_deceased', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych śmierci')
plt.ylabel('Nowe śmierci')

### 1.3 liczby nowych szczepień

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
sns.boxplot(data=part6, y='daily_persons_vaccinated', linewidth=2, ax=axes[0])
sns.boxplot(data=part6, y='daily_persons_fully_vaccinated', linewidth=2, ax=axes[1])
figure.suptitle('Wykres pudełkowy liczby nowych szczepień')

### 1.4 populacji

In [ ]:
sns.boxplot(data=part6, y='population', linewidth=2, log_scale=10)
plt.title('Wykres pudełkowy populacji w krajach na świecie')
plt.ylabel('Populacja')

### 1.5 PKB

In [ ]:
sns.boxplot(data=part6, y='gdp_usd', linewidth=2, log_scale=10)
plt.title('Wykres pudełkowy PKB na świecie [USD]')
plt.ylabel('PKB krajów w dolarach')

### 1.6 średnia długość życia w krajach

In [ ]:
sns.boxplot(data=part6, y='life_expectancy', linewidth=2)
plt.title('Wykres pudełkowy przewidywanej długości życia na świecie')
plt.ylabel('Przewidywana długości życia')

### 1.6 PKB na osobę

In [ ]:
sns.boxplot(data=part6, y='gdp_per_capita_usd', linewidth=2)
plt.title('Wykres pudełkowy PKB na osobę na świecie')
plt.ylabel('Wartość PKB na osobę')

### 1.6 cena benzyny w USA i UK

In [ ]:
sns.boxplot(data=data[-1], y='Gasoline Prices $/litre', x='CCA3', linewidth=2)
plt.title('Wykres pudełkowy ceny benzyny w USA i UK')
plt.xlabel('Kraj')
plt.ylabel('Cena benzyny $/litr')

### 1.6 giełdowa cena energi w Belgii

In [ ]:
sns.boxplot(data=data[-2], y='Stock Price', x='CCA3', linewidth=2)
plt.title('Wykres pudełkowy ceny energi w Belgii')
plt.xlabel('Kraj')
plt.ylabel('Cena benzyny $/litr')

### 1.6 liczba przestępstw z podziałem na jego rodzaj w Południowej Australi

In [ ]:
sns.boxplot(data=data[-3], x='Offence count', y='Offence Type', linewidth=2)
plt.title('Wykres pudełkowy przestępstw w Południowej Australi')

## <center>Część 2</center>
### Usuń wartości odstające dla co najmniej 2 informacji z części 1 poziomu 2. W tym celu użyj dwóch różnych metod obliczania outlierów. Uzasadnij swój wybór. Przeanalizuj otrzymane wyniki. 
<ol>
    <li style='font-size: 20px'>Metoda Z-Score</li>
    <p style='font-size: 20px'>Metoda statystyczna, która mierzy, jak wiele odchyleń standardowych wartość jest oddalona od średniej.<br>Wartości z wysokim Z-Score (np. powyżej 2 lub 3) mogą być uznane za wartości odstające, a następnie usunięte. </p>
    <br>
    <li style='font-size: 20px'>Metoda IQR (ang. InterQuartile Range)</li>
    <p style='font-size: 20px'>Technika oparta na kwartylach, w której oblicza się zakres międzykwartylowy (Q3 - Q1) i wartości odstające są określane jako wartości leżące poza granicami 1,5 IQR poniżej Q1 lub powyżej Q3. </p>
</ol>

In [ ]:
def del_outliers(series, method='zscore'):
    indexes = []
    if method == 'zscore':
        vec = zscore(series)
        for index in range(len(vec)):
            if -3 <= vec[index] >= 3:
                indexes.append(index)
    elif method == 'iqr':
        q1, q3, c = series.quantile(0.25), series.quantile(0.75), iqr(series) * 1.5
        for index, v in series.items():
            if q1 - c <= v >= q3 + c:
                indexes.append(index)
    else:
        raise ValueError('Nie ma takich rzeczy')
    return indexes

part6.drop(index=del_outliers(part6['daily_confirmed']), inplace=True)
part6.reset_index(drop=True, inplace=True)
part6.drop(index=del_outliers(part6['daily_deceased'], method='iqr'), inplace=True)
part6.reset_index(drop=True, inplace=True)
part6.drop(index=del_outliers(part6['daily_persons_vaccinated']), inplace=True)
part6.reset_index(drop=True, inplace=True)
part6.drop(index=del_outliers(part6['daily_persons_fully_vaccinated']), inplace=True)
part6.reset_index(drop=True, inplace=True)

### Wykresy pudełkowe po usunięciu <i>outlierów</i>

In [ ]:
sns.boxplot(data=part6, y='daily_confirmed', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych zachorowań')
plt.ylabel('Nowe zachorowania')

In [ ]:
sns.boxplot(data=part6, y='daily_deceased', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych śmierci')
plt.ylabel('Nowe śmierci')

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
sns.boxplot(data=part6, y='daily_persons_vaccinated', linewidth=2, ax=axes[0])
sns.boxplot(data=part6, y='daily_persons_fully_vaccinated', linewidth=2, ax=axes[1])
figure.suptitle('Wykres pudełkowy liczby nowych szczepień')

## <center>Część 3</center>
### Wykonaj następujące obliczenia:

### 3.1. średnia liczba nowych zachorowań, śmierci i szczepień dla co najmniej 10 wybranych krajów w wybranym miesiącu,

In [ ]:
def get_data_for_part_3(limit_to_countries=True, limit_to_month=None):
    df = part6[['country_name', 'date', 'daily_confirmed', 'daily_deceased', 'daily_persons_vaccinated'] + gdp_columns + population_columns]
    if limit_to_month is not None and 1 >= limit_to_month <= 12:
        df = df.loc[df['date'].dt.month == limit_to_month]
    if limit_to_countries is True:
        df = df[df['country_name'].isin(country_names['country_name'])]
    return df.reset_index(drop=True)

def plot_3_1():
    grouped = conf_dead_vacc.loc[:, ['country_name', 'daily_confirmed', 'daily_deceased', 'daily_persons_vaccinated']].groupby(['country_name'])
    avg = grouped.mean()
    avg.plot(kind='bar', title='Średnia liczba nowych zachorowań, śmierci i szczepień w styczniu dla 10 najbiedniejszych krajów', logy=True, xlabel='Nazwa kraju', ylabel='Wartość średniej', rot=45, figsize=(8, 7))

gdp_columns = [el for el in part6.columns if 'gdp' in el]
gdp_columns.remove('gdp_per_capita_usd')
population_columns = [el for el in part6.columns if 'Population' in el]
population_columns.remove('World Population Percentage')
population_columns.append('population')

country_names = part6[['country_name', 'gdp_usd']].drop_duplicates().sort_values(by='gdp_usd', ascending=False)[:10]  # wybieramy 10 najbiedniejszych krajów
country_names.reset_index(drop=True, inplace=True)
country_names.drop('gdp_usd', axis=1, inplace=True)
country_names.sort_values(by='country_name', ascending=True, inplace=True)
conf_dead_vacc = get_data_for_part_3(limit_to_month=1)

plot_3_1()

### 3.2. średnia zmiana liczby zachorowań, śmierci i szczepień dla co najmniej 10 wybranych krajów w wybranym miesiącu,

In [ ]:
def plot_3_2():
    avg_change = pd.DataFrame()
    for c in country_names['country_name']:
        tmp = conf_dead_vacc.loc[conf_dead_vacc['country_name'] == c, ['daily_confirmed', 'daily_deceased', 'daily_persons_vaccinated']].diff().mean().to_frame(c)
        avg_change = pd.concat([avg_change, tmp], axis=1)
    avg_change = avg_change.T
    avg_change.plot(kind='bar', title='Średnia zmiana liczby zachorowań, śmierci i szczepień w styczniu dla 10 najbiedniejszych krajów', logy=True, xlabel='Nazwa kraju', ylabel='Wartość średniej zmiany', rot=45, figsize=(8, 7))
    
plot_3_2()

### 3.3. mediana liczby nowych zachorowań, śmierci i szczepień dla co najmniej 10 wybranych krajów w wybranym miesiącu, 

In [ ]:
def plot_3_3():
    grouped = conf_dead_vacc.loc[:, ['country_name', 'daily_confirmed', 'daily_deceased', 'daily_persons_vaccinated']].groupby(['country_name'])
    median = grouped.median()
    median.plot(kind='bar', title='Mediana liczby nowych zachorowań, śmierci i szczepień w styczniu dla 10 najbiedniejszych krajów', logy=True, xlabel='Nazwa kraju', ylabel='Wartość mediany', rot=45, figsize=(8, 7))
    
plot_3_3()

### 3.4. odchylenie standardowe liczby nowych zachorowań, śmierci i szczepień dla co najmniej 10 wybranych krajów w wybranym miesiącu,

In [ ]:
def plot_3_4():
    grouped = conf_dead_vacc.loc[:, ['country_name', 'daily_confirmed', 'daily_deceased', 'daily_persons_vaccinated']].groupby(['country_name'])
    std = grouped.std()
    std.plot(kind='bar', title='Odchylenie standardowe liczby nowych zachorowań, śmierci i szczepień w styczniu dla 10 najbiedniejszych krajów', logy=True, xlabel='Nazwa kraju', ylabel='Wartość odchylenia standardowego', rot=45, figsize=(8, 7))
    
plot_3_4()

### 3.5. minimalna, średnia i maksymalna liczba nowych zachorowań, szczepień i śmierci, a PKB co najmniej 10 wybranych krajów,

In [ ]:
def plot_last(limit_to):
    df = get_data_for_part_3()
    
    fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(40, 20))
    fig.suptitle('Minimalna, średnia i maksymalna liczba nowych zachorowań, szczepień i śmierci, a PKB dla', fontsize=30)
    extra_data = df.groupby('country_name')[limit_to].last()
    extra_data.index = pd.MultiIndex.from_product([extra_data.index, [limit_to]])

    for i, country in enumerate(country_names['country_name']):
        country_stats = df[df['country_name'] == country][['daily_confirmed', 'daily_deceased', 'daily_persons_vaccinated']].describe().drop(index=['count', 'std', '25%', '50%', '75%'])
        country_stats = pd.concat([country_stats, extra_data[country]], axis=1)
        x, y = divmod(i, 5)
        country_stats.plot(kind='bar', ax=ax[x][y], grid=True, fontsize=15, rot=0, logy=True)
        ax[x][y].set_title(country, fontsize=15)
        if y == 0:
            ax[x][y].set_ylabel(f'Liczba zachorowań, szczepień, śmierci i wartość PKB dla {country}', fontsize=15)

plot_last('gdp_usd')

### 3.6. minimalna, średnia i maksymalna liczba nowych zachorowań, szczepień i śmierci, a populacja co najmniej 10 wybranych krajów.

In [ ]:
plot_last('population')

## <center>Część 4</center>
### Wykonaj normalizację dla następujących przypadków:

### 4.1. liczba zachorowań na 1000 osób,

In [ ]:
def normalize(before, num_of_people, ylabel, xlabel='date'):
    columns = [el for el in before.columns if ylabel in el]
    after = before.copy()
    after[columns] /= num_of_people
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    before.plot(x=xlabel, y=ylabel, kind='kde', ax=ax[0], title='Przed normalizacją', style='-', color='blue')
    after.plot(x=xlabel, y=ylabel, kind='kde', ax=ax[1], title='Po normalizacji', style='--', color='orange')
    
normalize(part6, 1000, 'daily_confirmed')

### 4.2. liczba szczepień na 1000 osób,

In [ ]:
normalize(part6, 1000, 'daily_persons_vaccinated')

### 4.3. liczba zgonów na 100 osób,

In [ ]:
normalize(part6, 100, 'daily_deceased')

### 4.4 wydatki na służbę zdrowia na 1000 osób,

In [ ]:
normalize(part6, 100, 'health_expenditure_usd', xlabel='country_name')

### 4.4 liczba zachorowań kobiet na 1000 osób,

In [ ]:
normalize(part6, 1000, 'daily_confirmed_female')

### 4.4 liczba zachorowań mężczyzn na 1000 osób,

In [ ]:
normalize(part6, 1000, 'daily_confirmed_male')

### 4.4 liczba zgonów kobiet na 100 osób,

In [ ]:
normalize(part6, 100, 'daily_deceased_female')

### 4.4 liczba zgonów mężczyzn na 100 osób,

In [ ]:
normalize(part6, 100, 'daily_deceased_male')

## <center>Część 5</center>
### Stwórz macierze korelacji, które przedstawią współczynniki korelacji dla każdej pary zmiennych:

### 5.1. liczba nowych zachorowań, szczepień i zgonów, a liczba nowych zachorowań, szczepień i zgonów (6 istotnych kombinacji),

In [ ]:
primary_cols = ['daily_confirmed', 'daily_deceased', 'daily_persons_vaccinated']
sns.heatmap(part6[primary_cols].corr(numeric_only=True), annot=True, fmt='.2f', cmap='viridis')

### 5.2. liczba nowych zachorowań, szczepień, zgonów, a łączna liczba zachorowań, szczepień, zgonów (9 istotnych kombinacji),

### 5.3. wskaźniki gospodarcze, a liczba nowych zachorowań, szczepień i zgonów (co najmniej 3 istotne kombinacje),

In [ ]:
cols = ['country_name', 'gdp_per_capita_usd', 'gdp_usd', 'health_expenditure_usd'] + primary_cols
sns.heatmap(part6[cols].corr(numeric_only=True), annot=True, fmt='.2f', cmap='viridis')

### 5.4. parametry demograficzne, a liczba nowych zachorowań, szczepień i zgonów (co najmniej 3 istotne kombinacje),

In [ ]:
cols = ['country_name', 'population', 'population_male', 'population_female', 'life_expectancy'] + primary_cols
sns.heatmap(part6[cols].corr(numeric_only=True), annot=True, fmt='.2f', cmap='viridis')

### 5.5 zamknięcie szkół, a liczba nowych zachorowań, śmierci i szczepień w <i>szkolnych</i> grupach wiekowych

In [ ]:
cols = ['school_closing', 'age_confirmed_0', 'age_confirmed_1', 'age_deceased_1', 'age_deceased_2']
sns.heatmap(part6[cols].corr(numeric_only=True), annot=True, fmt='.2f', cmap='viridis')

### 5.5 zachorowania i śmierci mężczyzn, a zachorowania i śmierci kobiet

In [ ]:
cols = ['daily_confirmed_male', 'daily_confirmed_female', 'daily_deceased_male', 'daily_deceased_female']
sns.heatmap(part6[cols].corr(), annot=True, fmt='.2f', cmap='viridis')

### 5.5 położenie kraju, a jego populacja i przewidywana długość życia

In [ ]:
cols = ['latitude', 'longitude', 'population', 'life_expectancy']
sns.heatmap(part6[cols].corr(), annot=True, fmt='.2f', cmap='viridis')

### 5.5 położenie kraju, a jego parametry gospodarcze

In [ ]:
cols = ['latitude', 'longitude', 'gdp_usd', 'gdp_per_capita_usd', 'health_expenditure_usd']
sns.heatmap(part6[cols].corr(), annot=True, fmt='.2f', cmap='viridis')